<a href="https://colab.research.google.com/github/Dibyajyoti227/datasciencecoursera/blob/master/Training_unevendim_images_week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow import keras

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [ ]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])
train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])
print('total number of horse training images:',len(os.listdir(train_horse_dir)))
print('total number of human training images:',len(os.listdir(train_human_dir)))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4

pic_index = 0

In [ ]:
fig = plt.gcf()
fig.set_size_inches(nrows*4,ncols*4)

pic_index += 8

next_horse_pix = [os.path.join(train_horse_dir,fname)
                  for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir,fname)
                  for fname in train_human_names[pic_index-8:pic_index]] 

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  sp = plt.subplot(nrows,ncols,i+1) 
  sp.axis('off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
model = tf.keras.models.Sequential([
                                 tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 tf.keras.layers.MaxPooling2D(2,2),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(128,activation = 'relu'),
                                 tf.keras.layers.Dense(1,activation = 'sigmoid')
                                 
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),metrics = ['accuracy'])

train_datagen = ImageDataGenerator(rescale=1/255)

train_dir = '/tmp/horse-or-human'

train_generator = train_datagen.flow_from_directory( 
    train_dir,
    target_size=(300,300),
    batch_size = 256, 
    class_mode = 'binary'                                              
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 5,
    epochs = 15,
    verbose=1
)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for k in uploaded.keys():
  path = '/content/'+k

  img = image.load_img(path, target_size = (300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  plt.imshow(img)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 10)

  print(classes)
  if classes[0]>0.5:
      print(k+"is a human")
  else:
      print(k+"is a horse")


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)